In [8]:
import jupyter_Importer
from MLModels_FeatureImportance import featureImportance_linearRegression
from MLModels_FeatureImportance import featureImportance_permutation
from MLModels_FeatureImportance import featureImportance_DTree
#from MLModels_FeatureImportance import df_2feature_be_generate

import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier    
from sklearn.model_selection import train_test_split

In [9]:
dataset = pd.read_csv("Occupancy.csv", names = ['x' + str(i) for i in range(1, 7)] + ['y'], sep = ",")
dataset.head()

,x1,x2,x3,x4,x5,x6,y
0,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-02 14:19:00,23.7,26.272,585.2,749.2,0.0047641630241641,1
2,2015-02-02 14:19:59,23.718,26.29,578.4,760.4,0.0047726609921251,1
3,2015-02-02 14:21:00,23.73,26.23,572.666666666667,769.666666666667,0.0047651525524654,1
4,2015-02-02 14:22:00,23.7225,26.125,493.75,774.75,0.0047437733559968,1


light+co2
co2+temperature

In [10]:
dataset_light_co2 = dataset.iloc[1:,3:5]
dataset_light_co2['y'] = dataset.iloc[1:,6:7]
dataset_light_co2.columns = ['x1','x2','y']
dataset_light_co2['x1'] = dataset_light_co2['x1'].apply(lambda x : float(x))
dataset_light_co2['x2'] = dataset_light_co2['x2'].apply(lambda x : float(x))
dataset_light_co2['y'] = dataset_light_co2['y'].apply(lambda x : int(x))
dataset_light_co2

,x1,x2,y
1,585.200000,749.200000,1
2,578.400000,760.400000,1
3,572.666667,769.666667,1
4,493.750000,774.750000,1
5,488.600000,779.000000,1
...,...,...,...
20556,429.750000,1505.250000,1
20557,423.500000,1514.500000,1
20558,423.500000,1521.500000,1
20559,418.750000,1632.000000,1


In [11]:
subset = dataset_light_co2.sample(frac=0.2,replace=False,random_state=43)
dataset_light_co2 = dataset_light_co2.append(subset)
dataset_light_co2 = dataset_light_co2.drop_duplicates(subset=['x1','x2','y'],keep=False)

subsample_x = subset.iloc[:,:-1].values
subsample_y = subset.iloc[:,-1].values

error_a = []
error_b = []
for i in range(21) :
    error_a.append(i)
    error_b.append(20-i)

print(dataset_light_co2.info(),subset.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6757 entries, 1 to 20560
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      6757 non-null   float64
 1   x2      6757 non-null   float64
 2   y       6757 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 211.2 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4112 entries, 8855 to 4674
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      4112 non-null   float64
 1   x2      4112 non-null   float64
 2   y       4112 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 128.5 KB
None None


In [12]:
importanceMeasure_x = dataset_light_co2.iloc[:,:-1]
importanceMeasure_y = dataset_light_co2.iloc[:,-1]

In [15]:
importanceReg,RegCoef = featureImportance_linearRegression(importanceMeasure_x,importanceMeasure_y)
model = KNeighborsClassifier(n_neighbors=5,algorithm="brute")  
model.fit(importanceMeasure_x,importanceMeasure_y)  
importancePermu,PermuCoef = featureImportance_permutation(model,importanceMeasure_x,importanceMeasure_y)
importanceDTree,DTreeCoef = featureImportance_DTree(importanceMeasure_x,importanceMeasure_y)
print("RegCoef : " +str(RegCoef))
print("importanceReg : "+str(importanceReg))
print("PermuCoef : "+str(PermuCoef))
print("importancePermu : "+str(importancePermu))
print("DTreeCoef : " +str(DTreeCoef))
print("importanceDTree : "+str(importanceDTree))

RegCoef : [0.0017731041814707073, 0.0002743642318678488]
importanceReg : [0.8659983079199367, 0.1340016920800632]
PermuCoef : [0.42956933550392196, 0.06124019535296734]
importancePermu : [0.8752261488360873, 0.12477385116391274]
DTreeCoef : [0.93720529546869, 0.06279470453130993]
importanceDTree : [0.93720529546869, 0.06279470453130993]


In [7]:
def df_2feature_be_generates(BE,i,n,importance=importanceReg) :#i for brute force or not,n for negative importance
    temp = dataset_light_co2.copy()
    df_2feature_be = pd.DataFrame.to_numpy(dataset_light_co2)
    
    if i >= 0 and n == 0 :
        e1 = temp['x1'].mean()*BE*(error_a[i]/(error_a[i]+error_b[i]))
        e2 = temp['x2'].mean()*BE*(error_b[i]/(error_a[i]+error_b[i]))
    elif i == -1 and n == 1:
        e1 = temp['x1'].mean()*BE*importance[0]
        e2 = temp['x2'].mean()*BE*importance[1]
    elif i == -1 and n == 0 :
        e1 = temp['x1'].mean()*BE*importance[1]
        e2 = temp['x2'].mean()*BE*importance[0]
    
    for idx,cell in temp['x1'].iteritems():
        temp['x1'][idx] += e1
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = dataset_light_co2.copy()

    for idx,cell in temp['x1'].iteritems():
        temp['x1'][idx] = temp['x1'][idx] - e1
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = dataset_light_co2.copy()

    for idx,cell in temp['x2'].iteritems():
        temp['x2'][idx] += e2
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = dataset_light_co2.copy()

    for idx,cell in temp['x2'].iteritems():
        temp['x2'][idx] = temp['x2'][idx] - e2
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = dataset_light_co2.copy()

    df_2feature_be = pd.DataFrame(df_2feature_be)
    #print(df_2feature_be)
    return df_2feature_be

In [8]:
#using importanceReg as error allocation rate to predict by svm
BE = 0.2
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceReg)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE = SVC(kernel='linear',C=1,gamma='auto')
clf_BE.fit(X_train, y_train)
y_BE_predict = clf_BE.predict(X_test)
result_BE_train = clf_BE.score(X_train, y_train)
result_BE_test = clf_BE.score(X_test, y_test)
subset_raw = clf_BE.score(subsample_x, subsample_y)
print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_train))
print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_test))
print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.08241831878934744),x2 with error (0.2%*0.9175816812106525) : 0.9728799763208524
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.08241831878934744),x2 with error (0.2%*0.9175816812106525) : 0.9738049430220512
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701


In [9]:
#using importancePermu as error allocation rate to predict by svm
df_2feature_be = df_2feature_be_generates(BE,-1,0,importancePermu)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE = SVC(kernel='linear',C=1,gamma='auto')
clf_BE.fit(X_train, y_train)
y_BE_predict = clf_BE.predict(X_test)
result_BE_train = clf_BE.score(X_train, y_train)
result_BE_test = clf_BE.score(X_test, y_test)
subset_raw = clf_BE.score(subsample_x, subsample_y)
print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_train))
print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_test))
print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))



d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.16415249588590236),x2 with error (0.2%*0.8358475041140976) : 0.9714370282669824
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.16415249588590236),x2 with error (0.2%*0.8358475041140976) : 0.9723249963001332
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9875972762645915


In [10]:
#brute force allocation
#brute force from 0-20 using svm
BE = 0.2
for i in range(21) :
    df_2feature_be = df_2feature_be_generates(BE,i,0)
    x_be = df_2feature_be.iloc[:, :-1].values
    y_be = df_2feature_be.iloc[:,-1].values
    X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
    clf_BE = SVC(kernel='linear',C=1,gamma='auto')
    clf_BE.fit(X_train, y_train)
    y_BE_predict = clf_BE.predict(X_test)
    result_BE_train = clf_BE.score(X_train, y_train)
    result_BE_test = clf_BE.score(X_test, y_test)
    subset_raw = clf_BE.score(subsample_x, subsample_y)
    print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_train))
    print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_test))
    print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))
    

d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.973175965665236
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9741009323664348
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9736199496818114
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9742489270386266
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

...

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.9),x2 with error (0.2%*0.1) : 0.966553204084653
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.9),x2 with error (0.2%*0.1) : 0.9656652360515021
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9873540856031129


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.9631123279561936
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.9632973212964333
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9851653696498055


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BESVM Accuracy (on training) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.9629643332840018
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.9622613585910907
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9846789883268483


In [11]:
#using importanceReg as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceReg)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE = KNeighborsClassifier()
KNN_BE.fit(X_train, y_train)
y_BE_predict = KNN_BE.predict(X_test)
result_BE_train = KNN_BE.score(X_train, y_train)
result_BE_test = KNN_BE.score(X_test, y_test)
subset_raw = KNN_BE.score(subsample_x, subsample_y)
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.08241831878934744),x2 with error (0.2%*0.9175816812106525) : 0.9794657392333876
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.08241831878934744),x2 with error (0.2%*0.9175816812106525) : 0.9748409057273938
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9883268482490273


In [12]:
#using importancePermu as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generates(BE,-1,0,importancePermu)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE = KNeighborsClassifier()
KNN_BE.fit(X_train, y_train)
y_BE_predict = KNN_BE.predict(X_test)
result_BE_train = KNN_BE.score(X_train, y_train)
result_BE_test = KNN_BE.score(X_test, y_test)
subset_raw = KNN_BE.score(subsample_x, subsample_y)
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.16415249588590236),x2 with error (0.2%*0.8358475041140976) : 0.977652804499038
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.16415249588590236),x2 with error (0.2%*0.8358475041140976) : 0.9749889003995856
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9883268482490273


In [13]:
#using importanceDTree as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceDTree)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE = KNeighborsClassifier()
KNN_BE.fit(X_train, y_train)
y_BE_predict = KNN_BE.predict(X_test)
result_BE_train = KNN_BE.score(X_train, y_train)
result_BE_test = KNN_BE.score(X_test, y_test)
subset_raw = KNN_BE.score(subsample_x, subsample_y)
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.04124195499916664),x2 with error (0.2%*0.9587580450008334) : 0.9823146366730797
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.04124195499916664),x2 with error (0.2%*0.9587580450008334) : 0.975432884416161
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9885700389105059


In [14]:
#using knn with brute force from 0-20
for i in range(21):
    df_2feature_be = df_2feature_be_generates(BE,i,0)
    x_be = df_2feature_be.iloc[:, :-1].values
    y_be = df_2feature_be.iloc[:,-1].values
    X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
    KNN_BE = KNeighborsClassifier()
    KNN_BE.fit(X_train,y_train)
    y_BE_predict = KNN_BE.predict(X_test)
    result_BE_train = KNN_BE.score(X_train, y_train)
    result_BE_test = KNN_BE.score(X_test, y_test)
    subset_raw = KNN_BE.score(subsample_x, subsample_y)
    
    print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_train))
    print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_test))
    print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))
    

d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9845345567559568
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.973952937694243
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9863813229571985


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9812416752996892
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9752848897439692
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9880836575875487


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

...

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.9),x2 with error (0.2%*0.1) : 0.9764688471215036
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.9),x2 with error (0.2%*0.1) : 0.970253070889448
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.9776158058309901
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.970993044250407
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9885700389105059


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.982869616693799
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.9712890335947906
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9861381322957199


In [15]:
from sklearn import preprocessing


In [16]:
#importanceReg,KMeans
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceReg)
x_be = df_2feature_be.iloc[:, :-1].values
x_be = preprocessing.scale(x_be)#標準化
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KM_BE=KMeans(n_clusters=2,init='random',random_state=5)
KM_BE.fit(X_train)
y_BE_predict = KM_BE.predict(X_train)
y_BE_predict_test = KM_BE.predict(X_test)
subsample_x = preprocessing.scale(subsample_x)
y_subset = KM_BE.predict(subsample_x)
result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    
    
print('BEKM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_train))
print('BEKM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_test))
print('BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))

d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.08241831878934744),x2 with error (0.2%*0.9175816812106525) : 0.9629273346159538
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.08241831878934744),x2 with error (0.2%*0.9175816812106525) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8959143968871596


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [17]:
#importancePermu,KMeans
df_2feature_be = df_2feature_be_generates(BE,-1,0,importancePermu)
x_be = df_2feature_be.iloc[:, :-1].values
x_be = preprocessing.scale(x_be)#標準化
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KM_BE=KMeans(n_clusters=2,init='random',random_state=5)
KM_BE.fit(X_train)
y_BE_predict = KM_BE.predict(X_train)
y_BE_predict_test = KM_BE.predict(X_test)
subsample_x = preprocessing.scale(subsample_x)
y_subset = KM_BE.predict(subsample_x)
result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    
    
print('BEKM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_train))
print('BEKM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_test))
print('BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))

d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.16415249588590236),x2 with error (0.2%*0.8358475041140976) : 0.9634453159686251
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.16415249588590236),x2 with error (0.2%*0.8358475041140976) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8959143968871596


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [18]:
#importanceDTree,KMeans
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceDTree)
x_be = df_2feature_be.iloc[:, :-1].values
#x_be = preprocessing.scale(x_be)#標準化
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KM_BE=KMeans(n_clusters=2,init='random',random_state=5)
KM_BE.fit(X_train)
y_BE_predict = KM_BE.predict(X_train)
y_BE_predict_test = KM_BE.predict(X_test)
#subsample_x = preprocessing.scale(subsample_x)
y_subset = KM_BE.predict(subsample_x)
result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    
    
print('BEKM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_train))
print('BEKM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_test))
print('BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))

d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.04124195499916664),x2 with error (0.2%*0.9587580450008334) : 0.28389077993192247
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.04124195499916664),x2 with error (0.2%*0.9587580450008334) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.2334630350194552


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [19]:
#using kMeans with brute force from 0-20
for i in range(21):
    df_2feature_be = df_2feature_be_generates(BE,i,0)
    x_be = df_2feature_be.iloc[:, :-1].values
    x_be = preprocessing.scale(x_be)#標準化
    y_be = df_2feature_be.iloc[:,-1].values
    X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
    KM_BE=KMeans(n_clusters=300,init='random',random_state=5)
    KM_BE.fit(X_train)
    y_BE_predict = KM_BE.predict(X_train)
    y_BE_predict_test = KM_BE.predict(X_test)
    subsample_x = preprocessing.scale(subsample_x)
    y_subset = KM_BE.predict(subsample_x)
    result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
    #print(score)
    result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
    subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    #print(score)
    #result_BE_train = KM_BE.score(X_train, y_train)
    #result_BE_test = KM_BE.score(X_test, y_test)
    #subset_raw = KM_BE.score(subsample_x, subsample_y)
    
    
    print('BEKM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_train))
    print('BEKM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_test))
    print('BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))
    

d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying t

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.005660796211336416
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.0


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison fail

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.00018499334023980296
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.0


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison fail

...

d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison fail

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.004587834837945781
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.0


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


BEKM Accuracy (on training) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.0025899067633564643
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.0


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
